COPYRIGHT © 2018 Kiran Arun <kironni@gmail.com>

### Setup

In [1]:
!rm -r Reinforcement_Learning-101-challenge
!git clone https://github.com/KiranArun/Reinforcement_Learning-101-challenge.git 1>/dev/null
!bash Reinforcement_Learning-101-challenge/scripts/setup.sh

rm: cannot remove 'Reinforcement_Learning-101-challenge': No such file or directory
Cloning into 'Reinforcement_Learning-101-challenge'...
remote: Counting objects: 79, done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 79 (delta 41), reused 50 (delta 20), pack-reused 0
Unpacking objects: 100% (79/79), done.
Extracting templates from packages: 100%


In [2]:
# DEFINITELY DO NOT EDIT THIS CELL

import os
# Start noVNC remote Xwindows in a browser with TurboVNC
!bash /content/Reinforcement_Learning-101-challenge/scripts/kill-novnc.sh >/dev/null
!sleep 2
# start novnc server and expose via ngrok NOT HTTPS
os.environ['PATH'] += "${PATH}:/opt/VirtualGL/bin:/opt/TurboVNC/bin"
# hack line below to get vnc to start as ipython subprocess
!timeout 5 /opt/websockify/run 5901 --web=/opt/noVNC --wrap-mode=ignore -- /opt/TurboVNC/bin/vncserver :1 -securitytypes otp -otp -noxstartup 2>/dev/null
get_ipython().system_raw('python /opt/websockify/run 5901 --web=/opt/noVNC --wrap-mode=ignore -- /opt/TurboVNC/bin/vncserver :1 -securitytypes otp -otp -noxstartup > /content/.vnc/stdout 2>&1 &')
# ngrok remote tunnel
get_ipython().system_raw('/content/ngrok http 5901 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print('XWindow Link:', json.load(sys.stdin)['tunnels'][0]['public_url'])"
# get Xwindows going (right click to get openbox menu)
get_ipython().system_raw('DISPLAY=:1 openbox &')
# one-time-password
!grep "one-time password:" /content/.vnc/stdout

kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]
XWindow Link: https://4b6c6a13.ngrok.io
Full control one-time password: 62618399


In [0]:
# DO NOT EDIT THIS CELL

# set up Xvfb and attach it's own vncserver and test with spinning gears
!pkill Xvfb   # clean-up
!pkill x11vnc # clean-up

get_ipython().system_raw('DISPLAY=:1 xterm &')
get_ipython().system_raw('/usr/bin/Xvfb :99 -screen 0 640x480x24 &')
get_ipython().system_raw('/usr/bin/x11vnc -rfbport 5902 -forever -display :99 &')
!sleep 2
get_ipython().system_raw('DISPLAY=:1 /usr/bin/vncviewer localhost:5902 &')
get_ipython().system_raw('DISPLAY=:99 glxgears &')

In [0]:
# open a terminal into the glxgears session using TightVNC
get_ipython().system_raw('DISPLAY=:99 xterm -e /bin/bash -l -c "pkill glxgears"')
get_ipython().system_raw('DISPLAY=:1 /usr/bin/vncviewer localhost:5902 &')
get_ipython().system_raw('DISPLAY=:99 xterm -geometry 105x35 &')

# RL Challenge

In [5]:
# importing libraries
import numpy as np
import matplotlib.pyplot as plt
import gym

import keras
import keras.layers as layers
from rl import agents,memory,policy

Using TensorFlow backend.


### Hyperparameters

In [0]:
# Hyperparams

# number of steps to keep in experience buffer
memory_limit = 400

# discount value
gamma = 0.99

# how much to update target graph
target_model_update = 1e-2

# learning rate
learning_rate = 1e-2

# number of steps to sample from buffer to train on
batch_size = 32

In [7]:
# DO NOT EDIT

# setting environment
env = gym.make('CartPole-v1')
num_actions = env.action_space.n

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


### Neural Network

In [8]:
keras.backend.clear_session()

# create neural net
nn = keras.models.Sequential()

# keep this as input layer
nn.add(layers.Flatten(input_shape=(1,) + env.observation_space.shape))

# keep this as output layer
nn.add(layers.Dense(num_actions))

nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


### Optimizer

https://keras.io/optimizers/

In [0]:
# set optimizer
optimizer = keras.optimizers.SGD(lr=learning_rate)

### Memory

In [0]:
# DO NOT EDIT

# set experience buffer
memory = memory.SequentialMemory(limit=memory_limit,
                                    window_length=1)

### Policy

choose from: https://github.com/keras-rl/keras-rl/blob/master/rl/policy.py

[info on choosing](https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-7-action-selection-strategies-for-exploration-d3a97b7cceaf)

In [0]:
# set policy
policy = policy.BoltzmannQPolicy()

### Agent

choose from: https://github.com/keras-rl/keras-rl/tree/master/rl/agents

and use: https://github.com/keras-rl/keras-rl/tree/master/examples

In [0]:
# create agent
model = agents.dqn.DQNAgent(model=nn,
                            nb_actions=num_actions,
                            memory=memory,
                            gamma=gamma,
                            batch_size=batch_size,
                            nb_steps_warmup=100,
                            target_model_update=target_model_update,
                            policy=policy)

In [0]:
# DO NOT EDIT

# compile model
model.compile(optimizer,
            metrics=['mae'])

In [14]:
# DO NOT EDIT

# train
history = model.fit(env,
                  nb_steps=1000,
                  verbose=2)

Training for 1000 steps ...
  12/1000: episode: 1, duration: 0.048s, episode steps: 12, steps per second: 251, episode reward: 12.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.083 [0.000, 1.000], mean observation: 0.123 [-1.985, 3.104], loss: --, mean_absolute_error: --, mean_q: --
  25/1000: episode: 2, duration: 0.009s, episode steps: 13, steps per second: 1429, episode reward: 13.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.077 [0.000, 1.000], mean observation: 0.104 [-2.166, 3.291], loss: --, mean_absolute_error: --, mean_q: --
  42/1000: episode: 3, duration: 0.011s, episode steps: 17, steps per second: 1482, episode reward: 17.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.706 [0.000, 1.000], mean observation: -0.052 [-2.212, 1.415], loss: --, mean_absolute_error: --, mean_q: --
  56/1000: episode: 4, duration: 0.010s, episode steps: 14, steps per second: 1369, episode reward: 14.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.286 [0.000, 1.000],

 725/1000: episode: 44, duration: 0.082s, episode steps: 15, steps per second: 183, episode reward: 15.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.800 [0.000, 1.000], mean observation: -0.081 [-2.861, 1.900], loss: 2.985959, mean_absolute_error: 5.350154, mean_q: 9.681129
 735/1000: episode: 45, duration: 0.050s, episode steps: 10, steps per second: 199, episode reward: 10.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.800 [0.000, 1.000], mean observation: -0.115 [-2.032, 1.201], loss: 3.331762, mean_absolute_error: 5.437999, mean_q: 9.862997
 761/1000: episode: 46, duration: 0.128s, episode steps: 26, steps per second: 203, episode reward: 26.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.692 [0.000, 1.000], mean observation: -0.001 [-2.790, 1.968], loss: 3.576557, mean_absolute_error: 5.532123, mean_q: 9.906427
 774/1000: episode: 47, duration: 0.074s, episode steps: 13, steps per second: 176, episode reward: 13.000, mean reward: 1.000 [1.000, 1.000], mean 

In [15]:
# DO NOT EDIT

# test
hist = model.test(env, nb_episodes=50, visualize=False)
print('mean:', np.mean(hist.history['episode_reward']))

Testing for 50 episodes ...
Episode 1: reward: 49.000, steps: 49
Episode 2: reward: 93.000, steps: 93
Episode 3: reward: 46.000, steps: 46
Episode 4: reward: 48.000, steps: 48
Episode 5: reward: 43.000, steps: 43
Episode 6: reward: 101.000, steps: 101
Episode 7: reward: 97.000, steps: 97
Episode 8: reward: 52.000, steps: 52
Episode 9: reward: 52.000, steps: 52
Episode 10: reward: 55.000, steps: 55
Episode 11: reward: 45.000, steps: 45
Episode 12: reward: 51.000, steps: 51
Episode 13: reward: 78.000, steps: 78
Episode 14: reward: 78.000, steps: 78
Episode 15: reward: 67.000, steps: 67
Episode 16: reward: 45.000, steps: 45
Episode 17: reward: 90.000, steps: 90
Episode 18: reward: 57.000, steps: 57
Episode 19: reward: 77.000, steps: 77
Episode 20: reward: 59.000, steps: 59
Episode 21: reward: 63.000, steps: 63
Episode 22: reward: 43.000, steps: 43
Episode 23: reward: 78.000, steps: 78
Episode 24: reward: 45.000, steps: 45
Episode 25: reward: 44.000, steps: 44
Episode 26: reward: 59.000, s

In [0]:
# save model to file
nn.save('/content/models/nn.hdf5', overwrite=True)